## Procesar los datos

In [2]:
import pandas as pd

In [3]:
senadores_path = 'data/resultados_elecciones_senadores_ce_1989_2017.xlsx'
presidenciales_path = 'data/resultados_elecciones_presidenciales_ce_1989_2017_Chile.xlsx'

senadores_df = pd.read_excel(senadores_path)
presidentes_df = pd.read_excel(presidenciales_path)

In [4]:

columns_to_keep = [
    "Tipo de Elección", "Cargo", "Fecha de Elección", "Año de Elección", 
    "Inicio de Período", "Fin de Período", "Id Región", "Región", "Comuna", 
    "Candidato (a)", "Electo(a)", "Partido", "Sigla Partido", "Votos Totales"
]

senadores_df_simplified = senadores_df[columns_to_keep]
presidentes_df_simplified = presidentes_df[columns_to_keep]

def clean_votes(votes):
    if isinstance(votes, str):
        return int(votes.replace(',', '').replace('.', ''))
    return votes

senadores_df_simplified['Votos Totales'] = senadores_df_simplified['Votos Totales'].apply(clean_votes)
presidentes_df_simplified['Votos Totales'] = presidentes_df_simplified['Votos Totales'].apply(clean_votes)

senadores_df_multiindex = senadores_df_simplified.set_index([
    "Año de Elección", "Candidato (a)", "Región", "Comuna"
])

presidentes_df_multiindex = presidentes_df_simplified.set_index([
    "Año de Elección", "Candidato (a)", "Región", "Comuna"
])

agg_funcs = {col: 'first' for col in senadores_df_multiindex.columns.difference(['Votos Totales'])}
agg_funcs['Votos Totales'] = 'sum'

senadores_df_grouped = senadores_df_multiindex.groupby(level=["Año de Elección", "Candidato (a)", "Región"]).agg(agg_funcs)
presidentes_df_grouped = presidentes_df_multiindex.groupby(level=["Año de Elección", "Candidato (a)", "Región"]).agg(agg_funcs)

senadores_df_grouped['Votacion Total'] = senadores_df_grouped.groupby(level=["Año de Elección", "Candidato (a)"])['Votos Totales'].transform('sum')
presidentes_df_grouped['Votacion Total'] = presidentes_df_grouped.groupby(level=["Año de Elección", "Candidato (a)"])['Votos Totales'].transform('sum')

senadores_df_grouped_reset = senadores_df_grouped.reset_index()
presidentes_df_grouped_reset = presidentes_df_grouped.reset_index()

senadores_df_grouped_reset = senadores_df_grouped.reset_index()
presidentes_df_grouped_reset = presidentes_df_grouped.reset_index()

senadores_df_grouped_reset['Electo_Sort'] = senadores_df_grouped_reset['Electo(a)'].apply(lambda x: 1 if x == 'SI' else 0)
presidentes_df_grouped_reset['Electo_Sort'] = presidentes_df_grouped_reset['Electo(a)'].apply(lambda x: 1 if x == 'SI' else 0)

senadores_df_sorted = senadores_df_grouped_reset.sort_values(
    by=['Fecha de Elección', 'Id Región', 'Electo_Sort', 'Votos Totales'],
    ascending=[True, True, False, False]
)

presidentes_df_sorted = presidentes_df_grouped_reset.sort_values(
    by=['Fecha de Elección', 'Id Región', 'Electo_Sort', 'Votos Totales'],
    ascending=[True, True, False, False]
)

senadores_df_sorted.drop('Electo_Sort', axis=1, inplace=True)
presidentes_df_sorted.drop('Electo_Sort', axis=1, inplace=True)


/tmp/ipykernel_63341/1334340909.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senadores_df_simplified['Votos Totales'] = senadores_df_simplified['Votos Totales'].apply(clean_votes)
/tmp/ipykernel_63341/1334340909.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  presidentes_df_simplified['Votos Totales'] = presidentes_df_simplified['Votos Totales'].apply(clean_votes)


## Objetos

In [5]:


# Definición de clases
class Candidato:
    def __init__(self, nombre, partido, votos_por_region, electo=None):
        self.nombre = nombre
        self.partido = partido
        self.votos_por_region = votos_por_region
        self.electo = electo


    def votos_totales(self):
        return sum(self.votos_por_region.values())

    def to_dict(self):
        # Ordenar los votos por región según el ID de la región
        votos_ordenados_por_region = dict(sorted(self.votos_por_region.items()))
        return {
            'nombre': self.nombre,
            'partido': self.partido,
            'votos_por_region': votos_ordenados_por_region,
            'votos_totales': self.votos_totales(),
            'electo': self.electo

        }

class Eleccion:
    def __init__(self, fecha, año, cargo, periodo_inicio, periodo_fin, tipo):
        self.fecha = fecha
        self.año = año
        self.cargo = cargo
        self.periodo_inicio = periodo_inicio
        self.periodo_fin = periodo_fin
        self.tipo = tipo
        self.candidatos = []

    def agregar_candidato(self, nombre, partido, votos_por_region, electo=None):
        candidato = Candidato(nombre, partido, votos_por_region, electo)
        self.candidatos.append(candidato)

    def to_dict(self):
        return {
            'fecha': self.fecha,
            'año': self.año,
            'cargo': self.cargo,
            'periodo_inicio': self.periodo_inicio,
            'periodo_fin': self.periodo_fin,
            'tipo': self.tipo,
            'candidatos': [candidato.to_dict() for candidato in self.candidatos]
        }

class EleccionPresidencial(Eleccion):
    def __init__(self, fecha, año, cargo, periodo_inicio, periodo_fin, tipo):
        super().__init__(fecha, año, cargo, periodo_inicio, periodo_fin, tipo)
        self.votos_totales_nacionales = 0

    def agregar_candidato(self, nombre, partido, votos_por_region):
        # Agregar los votos totales por región al total nacional
        self.votos_totales_nacionales += sum(votos_por_region.values())
        super().agregar_candidato(nombre, partido, votos_por_region)

    def to_dict(self):
        # Incluir la votación total nacional en la representación del diccionario
        eleccion_dict = super().to_dict()
        eleccion_dict['votos_totales_nacionales'] = self.votos_totales_nacionales
        return eleccion_dict

# Clase para elecciones de senadores

class EleccionSenadores(Eleccion):

    def to_dict(self):
        eleccion_dict = super().to_dict()
        return eleccion_dict



## Funciones

In [12]:


def eleccion_presidencial_con_votos_por_region(año):
    # Filtrar el DataFrame presidencial por el año dado
    presidenciales_año_df = presidentes_df_sorted[presidentes_df_sorted['Año de Elección'] == año]

    # Crear un objeto de elección presidencial
    fecha_eleccion = presidenciales_año_df['Fecha de Elección'].iloc[0].strftime('%d/%m/%Y')
    inicio_periodo = presidenciales_año_df['Inicio de Período'].iloc[0]
    fin_periodo = presidenciales_año_df['Fin de Período'].iloc[0]
    eleccion_presidencial = EleccionPresidencial(
        fecha=fecha_eleccion,
        año=año,
        cargo='PRESIDENTE',
        periodo_inicio=inicio_periodo,
        periodo_fin=fin_periodo,
        tipo='Presidencial'
    )

    # Agregar candidatos con votos por región al objeto de elección presidencial
    for candidato in presidenciales_año_df['Candidato (a)'].unique():
        votos_por_region = presidenciales_año_df[presidenciales_año_df['Candidato (a)'] == candidato]
        votos_por_region_dict = votos_por_region.set_index('Id Región')['Votos Totales'].to_dict()
        partido = votos_por_region['Partido'].iloc[0]
        eleccion_presidencial.agregar_candidato(
            nombre=candidato,
            partido=partido,
            votos_por_region=votos_por_region_dict
        )

    # Convertir el objeto de elección presidencial a un diccionario y retornarlo
    return eleccion_presidencial.to_dict()


# Función que procesa los datos de los senadores y crea un objeto de EleccionSenadores
def eleccion_senadores_con_votos_por_region(año):
    # Filtrar el DataFrame de senadores por el año dado
    senadores_año_df = senadores_df[senadores_df['Año de Elección'] == año]

    # Crear un objeto de elección de senadores
    fecha_eleccion = senadores_año_df['Fecha de Elección'].iloc[0]
    inicio_periodo = senadores_año_df['Inicio de Período'].iloc[0]
    fin_periodo = senadores_año_df['Fin de Período'].iloc[0]
    eleccion_senadores = EleccionSenadores(
        fecha=fecha_eleccion,
        año=año,
        cargo='SENADOR',
        periodo_inicio=inicio_periodo,
        periodo_fin=fin_periodo,
        tipo='Parlamentaria'
    )

    # Agregar candidatos con votos por región al objeto de elección de senadores
    for _, row in senadores_año_df.iterrows():
        votos_por_region = {row['Id Región']: row['Votos Totales']}
        electo = row['Electo(a)'] == 'SI'
        eleccion_senadores.agregar_candidato(
            nombre=row['Candidato (a)'],
            partido=row['Partido'],
            votos_por_region=votos_por_region,
            electo=electo
        )

    # Convertir el objeto de elección de senadores a un diccionario y retornarlo
    return eleccion_senadores.to_dict()


## API

In [19]:
!pip -q install flask-ngrok
!pip -q install flask
!pip -q install ngrok


In [ ]:
import socket
from contextlib import closing
from flask import Flask, jsonify

In [24]:


app = Flask(__name__)

# Rutas de la API
@app.route('/eleccion/presidencial/<int:anio>', methods=['GET'])
def get_eleccion_presidencial(anio):
    try:
        resultado = eleccion_presidencial_con_votos_por_region(anio)
        return jsonify(resultado), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/eleccion/senadores/<int:anio>', methods=['GET'])
def get_eleccion_senadores(anio):
    try:
        resultado = eleccion_senadores_con_votos_por_region(anio)
        return jsonify(resultado), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

def run_server(app):
    ports = [5000, 5001, 5002, 5003]  # Lista de puertos para intentar
    for port in ports:
        try:
            app.run(host='127.0.0.1', port=port, debug=True)
            break  # Si el servidor se inicia correctamente, sal del bucle
        except OSError as e:
            if e.errno == 98:  # Error de puerto en uso
                print(f"El puerto {port} está en uso, intentando con otro puerto.")
            else:
                raise  # Si es otro tipo de OSError, lánzalo


In [45]:
import socket
from contextlib import closing

def find_free_port():
    with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
        s.bind(('', 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.getsockname()[1]

def run_server(app):
    try:
        port = find_free_port()
        app.run(host='127.0.0.1', port=port, debug=True)
    except OSError as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    run_server(app)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:41587
Press CTRL+C to quit
 * Restarting with stat
/home/ig/.local/lib/python3.11/site-packages/traitlets/traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/home/ig/.local/lib/python3.11/site-packages/traitlets/traitlets.py:2499: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use '564e2af5-175f-42bb-a07c-2ae99ea5fef7' instead of 'b"564e2af5-175f-42bb-a07c-2ae99ea5fef7"'.
  warn(
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/ig/.local/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/ig/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(

SystemExit: 1

In [17]:
!lsof -i :5000
